In [83]:
from datasets import load_from_disk
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
import pandas as pd
import numpy as np
import json

In [17]:
# Setup OpenAI API
load_dotenv()
chat = ChatOpenAI(temperature=0)

In [91]:
df = pd.DataFrame(load_from_disk("data/processed/ft_dataset.hf", keep_in_memory=True))

def get_chunks(series: pd.Series, max_chunks: int = 4):
    """
    Funktion zum Aufteilen von Texten in der Series in Chunks.
    Jeder Text wird zuerst in Sätze aufgeteilt und dann in Chunks unterteilt.
    
    :param series: pandas Series mit Texten.
    :param max_chunks: Maximale Anzahl von Chunks, in die ein Text aufgeteilt werden kann.
    :return: Eine neue Series, wobei jeder Eintrag eine Liste von Chunks ist.
    """
    
    def split_text(text:str, num_chunks:int):
        return [" ".join(splits) for splits in np.array_split([sentence+"." for sentence in text.split(". ")], num_chunks)] #get chunks
    
    chunksize = np.random.choice(list(range(1, max_chunks+1)), size=len(series))
    
    return [split_text(text, chunksize[i]) for i, text in enumerate(series)]

def get_source(text:str):
    template = (
        """Nachfolgend erhälst du einen Text. Bitte erstelle zu diesem Textabschnitt eine mögliche Quelle.
        Die Quelle soll möglichst kurz und ohne Zusatzinformationen sein.
        Beispiele: 
        - Wikipedia
        - Website xy
        - (Weitere eigene Quellen)
        """
    )

    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    human_template = "{message}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    messages = chat_prompt.format_prompt(message=text).to_messages()

    return chat(messages).content

def gpt_formatter(series: pd.Series):
    formatted = []
    for chunks in series:
        formatted.append(json.dumps([{"SOURCE":get_source(text), "CONTENT":text} for text in chunks], ensure_ascii=False))

    return formatted

df["context"] = get_chunks(df.context)
gpt_formatter(df.context[:2])

['[{"SOURCE": "Wikipedia", "CONTENT": "Während einige Götter Krankheit und Tod bringen, heilen andere Kranke und beschützen die Toten, und andere Götter wiederum vereinen diese beiden Funktionen. Bekannt ist der griechische Gott Asklepios der Medizin und Heilkunst. In China wurde der Arzt Baosheng Dadi nach seinem Tod zum Gott der Medizin erhoben. Zu den Göttern, die Krankheiten herbeiführen, zählen Pakoro Kamui bei den Ainu sowie Lugal-Irra und Namtar in Mesopotamien. Letzterem wurde nachgesagt, 60 verschiedene Krankheiten verursachen zu können. In den Veden bringt Rudra oft Krankheit und Verwüstung, wird aber auch als Heiler verehrt. Die Eigenschaften, die Totengöttern zugeschrieben werden, hängen von den religiös-kulturellen Vorstellungen des Geschehens nach dem Tode ab. Die ägyptische Göttin Hathor behütet die Toten, und im Hinduismus richtet Yama über die Toten.."}]',
 '[{"SOURCE": "Quelle: Wikipedia", "CONTENT": "Ein auf eine Gleichspannung  aufgeladener realer Kondensator entläd

In [81]:
get_source(df.context[3][0])

AIMessage(content='Quelle: Wikipedia')